### Libraries

In [28]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
import numpy as np
from sklearn.metrics import mean_absolute_error

### Data Loading

In [20]:
def_feature = pd.read_csv("input/Xente_Variable_Definitions.csv")
data = pd.read_csv("input/training.csv")
X_test = pd.read_csv("input/test.csv")
sample_submission = pd.read_csv("input/sample_submission.csv")

data = data.dropna(axis=0) #Drop observations/rows with missing values
y = data.FraudResult #The target label
X = data.copy()
X.drop(['FraudResult'], axis=1, inplace=True) #Only the features data
cols_cat = list(data.dtypes[data.dtypes=="object"].index) #liste des feature categorical

In [7]:
data.columns

Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId',
       'ProductCategory', 'ChannelId', 'Amount', 'Value',
       'TransactionStartTime', 'PricingStrategy', 'FraudResult'],
      dtype='object')

In [8]:
X.describe()

,CountryCode,Amount,Value,PricingStrategy
count,95662.0,9.566200e+04,9.566200e+04,95662.000000
mean,256.0,6.717846e+03,9.900584e+03,2.255974
std,0.0,1.233068e+05,1.231221e+05,0.732924
min,256.0,-1.000000e+06,2.000000e+00,0.000000
25%,256.0,-5.000000e+01,2.750000e+02,2.000000
50%,256.0,1.000000e+03,1.000000e+03,2.000000
75%,256.0,2.800000e+03,5.000000e+03,2.000000
max,256.0,9.880000e+06,9.880000e+06,4.000000


In [9]:
X.describe(include=object, exclude=[np.int64, np.float64])

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,ProviderId,ProductId,ProductCategory,ChannelId,TransactionStartTime
count,95662,95662,95662,95662,95662,95662,95662,95662,95662,95662,95662
unique,95662,94809,3633,3627,3742,1,6,23,9,4,94556
top,TransactionId_76871,BatchId_67019,AccountId_4841,SubscriptionId_3829,CustomerId_7343,UGX,ProviderId_4,ProductId_6,financial_services,ChannelId_3,2018-12-24T16:30:13Z
freq,1,28,30893,32630,4091,95662,38189,32635,45405,56935,17


# First Model
Very simple model with no feature engineering to have a reference of performance when we improve out model. The missing values are droped for simplicity. At first, the categorical values are also droped.

In [22]:
y1 = y.copy()
X1 = X.copy()
X1.drop(cols_cat, axis=1, inplace=True) #Delete categorical data
X1.head()

,CountryCode,Amount,Value,PricingStrategy
0,256,1000.0,1000,2
1,256,-20.0,20,2
2,256,500.0,500,2
3,256,20000.0,21800,2
4,256,-644.0,644,2


In [30]:
first_model_decision_tree = DecisionTreeRegressor(random_state=1)
first_model_decision_tree.fit(X1, y1)
print("Making predictions for the following 5 data:")
print(X1.head())
print(y1.head())
print("The predictions are")
predicted_first_model = first_model_decision_tree.predict(X1)
print(predicted_first_model)
print(mean_absolute_error(y, predicted_first_model))

Making predictions for the following 5 data:
   CountryCode   Amount  Value  PricingStrategy
0          256   1000.0   1000                2
1          256    -20.0     20                2
2          256    500.0    500                2
3          256  20000.0  21800                2
4          256   -644.0    644                2
0    0
1    0
2    0
3    0
4    0
Name: FraudResult, dtype: int64
The predictions are
[0.         0.         0.00023132 ... 0.         0.         0.        ]
0.000595960685908906


In [33]:
df_summary = X1.head()
df_summary

,CountryCode,Amount,Value,PricingStrategy
0,256,1000.0,1000,2
1,256,-20.0,20,2
2,256,500.0,500,2
3,256,20000.0,21800,2
4,256,-644.0,644,2


### Model Validation